In [5]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from src.arch import NN
from src import setup
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
from pathlib import Path


In [6]:
#load data
raw_train = pd.read_csv(setup.raw_train_path) 
train_demographics = pd.read_csv(setup.train_demographics_path)
raw_test = pd.read_csv(setup.raw_test_path)
test_demographics = pd.read_csv(setup.test_demographics_path)

In [ ]:
# making 1D data lists for each sensor
gestures = raw_train['gesture'].unique()
subjects = raw_train['subject'].unique()

#sensor data lists
svm_data = []
thermo_data = []
tof_data = []
gyro_data = []

#demographics data
shoulder_to_wrist_cm = []
elbow_to_wrist_cm = []
height = []
handedness = []
adult_child = []
sex = []
age = []


#labels for each gesture and subject
labels = []
for gesture in gestures:   
    for subject in subjects:
        subject_data = raw_train[(raw_train['gesture'] == gesture) & (raw_train['subject'] == subject)]
        subject_demographics = train_demographics[train_demographics['subject'] == subject]

        
        #calculating svm (sensor vector magnitude) from accelerometer data
        svm = np.sqrt(subject_data['acc_x']**2 + subject_data['acc_y']**2 + subject_data['acc_z']**2)
        svm_data.append(svm.values)

        thermo_data.append(subject_data[['thm_1', 'thm_2', 'thm_3', 'thm_4', 'thm_5']].values)
        gyro_data.append(subject_data[['rot_w', 'rot_x', 'rot_y', 'rot_z']].values)
        labels.append(gesture)

        #adding all tof data
        tof_cols = [f'tof_{sensor}_v{pixel}' for sensor in range(1, 6) for pixel in range(64)]
        tof_data.append(subject_data[tof_cols].values)

        #adding demographics data
        shoulder_to_wrist_cm.append(subject_demographics['shoulder_to_wrist_cm'].values[0])
        elbow_to_wrist_cm.append(subject_demographics['elbow_to_wrist_cm'].values[0])
        height.append(subject_demographics['height_cm'].values[0])
        handedness.append(subject_demographics['handedness'].values[0])
        adult_child.append(subject_demographics['adult_child'].values[0])
        sex.append(subject_demographics['sex'].values[0])
        age.append(subject_demographics['age'].values[0])


#converting lists to numpy arrays
svm_data = np.array(svm_data)
thermo_data = np.array(thermo_data)
tof_data = np.array(tof_data)
gyro_data = np.array(gyro_data)
shoulder_to_wrist_cm = np.array(shoulder_to_wrist_cm)
elbow_to_wrist_cm = np.array(elbow_to_wrist_cm)
height = np.array(height)
handedness = np.array(handedness)
adult_child = np.array(adult_child)
sex = np.array(sex)
age = np.array(age)
        

In [ ]:
#splitting into train and val sets